In [16]:
import pandas as pd
migraine_data=pd.read_excel('Migraine_PatientData.xlsx')


In [17]:
migraine_data.columns

Index(['Age', 'Duration', 'Frequency', 'Location', 'Character', 'Intensity',
       'Nausea', 'Vomit', 'Phonophobia', 'Photophobia', 'Visual', 'Sensory',
       'Dysphasia', 'Dysarthria', 'Vertigo', 'Tinnitus', 'Hypoacusis',
       'Diplopia', 'Defect', 'Ataxia', 'Conscience', 'Paresthesia', 'DPF',
       'Stress'],
      dtype='object')

In [18]:
in_features=["Age", "Duration", "Frequency", "Location", "character", "Intensity", "Nausea", "Vomit", "Phonophobia", "Photophobia", "Visual", "Sensory", "Dysphasia", "Dysarthria",
             "Vertigo", "Tinnitus", "Hypoacusis", "Diplopia", "Defect", "Ataxia", "Conscience", "Paresthesia", "DPF"]
out_feature="Stress"


In [8]:
#whereas the input parameters are advised to be normalised or standardized, but here in this datasets we are dealing with patient age, intensity of a symptom(feature).
#so I prefer not to normalize it. 
#moving towards the learning model directly

In [19]:
migraine_data

,Age,Duration,Frequency,Location,Character,Intensity,Nausea,Vomit,Phonophobia,Photophobia,...,Vertigo,Tinnitus,Hypoacusis,Diplopia,Defect,Ataxia,Conscience,Paresthesia,DPF,Stress
0,30,1,5,1,1,2,1,0,1,1,...,NaN,0,0,0,0,0,0,0,0,0.761905
1,50,3,5,1,1,3,1,1,1,1,...,1.0,0,0,0,0,0,0,0,0,1.000000
2,53,2,1,1,1,2,1,1,1,1,...,0.0,0,0,0,0,0,0,0,0,0.619048
3,45,3,5,1,1,3,1,0,1,1,...,1.0,0,0,0,0,0,0,0,0,1.000000
4,53,1,1,1,1,2,1,0,1,1,...,0.0,0,0,0,0,0,0,0,1,0.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,39,1,1,1,1,3,1,1,1,1,...,1.0,0,0,1,0,0,0,0,1,0.000000
396,20,3,2,1,1,3,1,0,1,1,...,0.0,1,1,0,0,0,0,0,1,0.000000
397,28,3,2,1,1,3,1,0,1,1,...,1.0,0,0,0,1,0,0,0,1,0.000000
398,20,3,1,1,1,3,1,0,1,1,...,1.0,0,0,0,0,0,0,1,1,0.000000


In [41]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras.callbacks import EarlyStopping, LambdaCallback
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [42]:
import numpy as np

In [43]:
import matplotlib.pyplot as plt
import cv2



In [63]:
#normalize the datasets


df = migraine_data.iloc[:, :23]
in_features = (df - df.mean())/df.std()
in_features.head()

,Age,Duration,Frequency,Location,Character,Intensity,Nausea,Vomit,Phonophobia,Photophobia,...,Dysarthria,Vertigo,Tinnitus,Hypoacusis,Diplopia,Defect,Ataxia,Conscience,Paresthesia,DPF
0,-0.140456,-0.791217,1.572246,0.102541,0.080986,-0.611589,0.112368,-0.689075,0.151527,0.142678,...,-0.05,NaN,-0.25233,-0.123249,-0.070799,-0.123249,NaN,-0.133294,-0.08682,-0.832573
1,1.507120,1.802937,1.572246,0.102541,0.080986,0.689664,0.112368,1.447592,0.151527,0.142678,...,-0.05,2.638656,-0.25233,-0.123249,-0.070799,-0.123249,NaN,-0.133294,-0.08682,-0.832573
2,1.754257,0.505860,-0.814465,0.102541,0.080986,-0.611589,0.112368,1.447592,0.151527,0.142678,...,-0.05,-0.378031,-0.25233,-0.123249,-0.070799,-0.123249,NaN,-0.133294,-0.08682,-0.832573
3,1.095226,1.802937,1.572246,0.102541,0.080986,0.689664,0.112368,-0.689075,0.151527,0.142678,...,-0.05,2.638656,-0.25233,-0.123249,-0.070799,-0.123249,NaN,-0.133294,-0.08682,-0.832573
4,1.754257,-0.791217,-0.814465,0.102541,0.080986,-0.611589,0.112368,-0.689075,0.151527,0.142678,...,-0.05,-0.378031,-0.25233,-0.123249,-0.070799,-0.123249,NaN,-0.133294,-0.08682,1.198093


In [64]:
out_feature=migraine_data.iloc[:,-1]
out_feature.head()

0    0.761905
1    1.000000
2    0.619048
3    1.000000
4    0.666667
Name: Stress, dtype: float64

In [65]:
x_arr = in_features.values
y_arr = out_feature.values


In [66]:
x_arr

array([[-0.14045588, -0.79121691,  1.5722458 , ..., -0.13329357,
        -0.08682041, -0.8325731 ],
       [ 1.50712045,  1.8029369 ,  1.5722458 , ..., -0.13329357,
        -0.08682041, -0.8325731 ],
       [ 1.7542569 ,  0.50585999, -0.81446509, ..., -0.13329357,
        -0.08682041, -0.8325731 ],
       ...,
       [-0.30521352,  1.8029369 , -0.21778737, ..., -0.13329357,
        -0.08682041,  1.198093  ],
       [-0.96424405,  1.8029369 , -0.81446509, ..., -0.13329357,
        11.48923409,  1.198093  ],
       [ 0.43619583, -0.79121691, -0.81446509, ...,  7.48348181,
        -0.08682041,  1.198093  ]])

In [67]:
y_arr

array([0.76190476, 1.        , 0.61904762, 1.        , 0.66666667,
       0.47619048, 0.        , 0.71428571, 0.95238095, 0.61904762,
       0.80952381, 0.76190476, 0.9047619 , 0.57142857, 0.9047619 ,
       0.95238095, 0.85714286, 0.66666667, 0.76190476, 0.95238095,
       0.76190476, 0.9047619 , 0.66666667, 0.76190476, 0.76190476,
       0.80952381, 0.80952381, 0.85714286, 0.71428571, 0.66666667,
       0.9047619 , 0.9047619 , 0.85714286, 0.66666667, 0.71428571,
       0.61904762, 0.76190476, 0.80952381, 0.76190476, 0.95238095,
       0.71428571, 0.66666667, 0.66666667, 0.71428571, 0.80952381,
       0.85714286, 0.76190476, 0.80952381, 0.76190476, 0.71428571,
       0.80952381, 0.80952381, 0.80952381, 0.71428571, 0.71428571,
       0.9047619 , 0.71428571, 0.71428571, 0.71428571, 0.66666667,
       0.80952381, 0.66666667, 0.80952381, 0.76190476, 0.66666667,
       0.61904762, 0.66666667, 0.85714286, 0.80952381, 0.71428571,
       0.71428571, 0.57142857, 0.52380952, 0.66666667, 0.61904

In [68]:
#train and test split

x_train,x_test,y_train,y_test = train_test_split(x_arr,y_arr,test_size=0.08,random_state=1)
print('Training data: ',x_train.shape,y_train.shape)
print('Test data :',x_test.shape,y_test.shape)

Training data:  (368, 23) (368,)
Test data : (32, 23) (32,)


In [69]:

def surrogate_model():
  ann_model=keras.Sequential([Dense(10,input_shape=(23, ),activation = 'relu'),
                    Dense(20,activation='relu'),Dense(1,activation='relu'),Dense(1)])
  ann_model.compile(loss='mse',optimizer ='adam')
  return ann_model
surrogate_model().summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_40 (Dense)            (None, 10)                240       
                                                                 
 dense_41 (Dense)            (None, 20)                220       
                                                                 
 dense_42 (Dense)            (None, 1)                 21        
                                                                 
 dense_43 (Dense)            (None, 1)                 2         
                                                                 
Total params: 483
Trainable params: 483
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from tensorflow.keras import optimizers

#stopper = EarlyStopping(monitor= 'val_loss', patience = 5)
model = surrogate_model()
prediction_untrainedData = model.predict(x_test)   
model.compile(optimizers.Adam(learning_rate=0.001), loss='mean_squared_error', metrics=['mse'])
history=model.fit(x_train,y_train,validation_data =(x_test,y_test), epochs = 10)

In [ ]:
prediction_trainedData = model.predict(x_test)
model.compare.predictions(prediction_untrainedData,prediction_trainedData,y_test)   #comparison of prediction

In [ ]:
#plotting

plt.subplot(300)
#plt.plot(history.history['val_loss'], label='test')
plt.title('surrogate_model_loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right')
plt.show()